In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as mpl
from datetime import timedelta

from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from generate_results_utils import *

colors = ["#FF0000", "#18E3FF", "#FFA90C", "#D9E501", "#0A2DC2", "#05A41B", "#878787", "#935948", "#b372fa", "#FC12DD"]
bench_colors = ["#000000", "#FF0000", "#18E3FF", "#FFA90C", "#05A41B"]


In [ ]:
# strategy_name = 'A2C/EVAL_A2C_V213_12201326_200M/all'
# strategy_name = 'cs_eval/A2C/EVAL_A2C_V221_2301221555_1D_mini_200/all'
strategy_name = 'ce/ce_12h_2301132144_1/all'

strat_name_clean = 'Ensemble 12h'
# strategy_name = 'A2C/A2C_V213_12181020_220M'
# portfolio_prefix = f'../results/cs/{strategy_name}'
# portfolio_prefix = f'../results/cs_eval/{strategy_name}'
portfolio_prefix = f'../results/{strategy_name}'
res = '12h'

In [ ]:
df_value = get_portfolio_value(portfolio_prefix)
df_actions = get_actions(portfolio_prefix)
df_prices = get_prices(df_actions.index[0], df_actions.index[-1], res)
df_benchmark = get_benchmark(df_actions.index[0], df_actions.index[-1], res)
df_benchmark = df_benchmark[["BAH"]]

df_returns = merge_strategy_and_benchmark_returns(df_value, df_benchmark)
df_returns = get_cumulative_returns(df_returns)

In [ ]:
names = df_prices.columns

fig = make_subplots(specs=[[{"secondary_y": True}]])

for idx, name in enumerate(names):
    fig.add_trace(go.Scatter(x=df_prices.index,y=df_prices[name],name=name, marker_color=colors[idx]),secondary_y=False)  #
    sell_idx = df_actions[df_actions[name] < 0][name].index
    buy_idx = df_actions[df_actions[name] > 0][name].index
    fig.add_trace(go.Scatter(x=buy_idx,y=df_prices.loc[buy_idx][name],
                             name='buy', mode = 'markers', marker_color='rgba(50, 205, 50, .9)',
                             marker_size=5, showlegend=False),
                  secondary_y=False)
    fig.add_trace(go.Scatter(x=sell_idx,y=df_prices.loc[sell_idx][name],
                             name=f'sell', mode = 'markers', marker_color='rgba(255, 0, 0, .9)',
                             marker_size=5, showlegend=False),
                  secondary_y=False)

fig.update_layout(title_text='Buy / Sell Actions', height=800, template='plotly_white')
fig.write_html(f"{portfolio_prefix}_actions_overview.html")
fig.show()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# for idx, name in enumerate(names):
#     fig.add_trace(go.Scatter(x=df_prices.index,y=df_prices[name],name=name, marker_color="#ccc"),secondary_y=False) 
    
for idx, name in enumerate(df_returns.columns):
    if idx == 0:
        opacity = 1
        legend_name = strat_name_clean
    else:
        opacity = 0.8
        legend_name = name
    fig.add_trace(go.Scatter(x=df_returns.index,y=df_returns[name],name=legend_name, marker_color=bench_colors[idx], opacity=opacity),secondary_y=False)
    fig.update_traces(connectgaps=True)

fig.update_layout(height=600, template='plotly_white')
fig.show()
filename = f"{portfolio_prefix}_performance_compare"
fig.write_html(f"{filename}.html")
fig.write_image(f"{filename}.png", width=1200, scale=2)